In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import scipy.optimize as opt

In [2]:


ticker = 'AAPL'
data = yf.Ticker(ticker)
df = data.history(period='1d', start='2020-1-1', end='2020-12-31')

start_date = '2020-01-01'
end_date = '2023-12-31'
quality = yf.Ticker('SPHQ').history(period='1d', start=start_date, end=end_date)['Close'].pct_change()
value = yf.Ticker('IVE').history(period='1d', start=start_date, end=end_date)['Close'].pct_change()
dividend = yf.Ticker('SPYD').history(period='1d', start=start_date, end=end_date)['Close'].pct_change()
low_vol = yf.Ticker('LOWV.L').history(period='1d', start=start_date, end=end_date)['Close'].pct_change()
momentum = yf.Ticker('SPMO').history(period='1d', start=start_date, end=end_date)['Close'].pct_change()

quality.index = pd.to_datetime(quality.index).strftime('%Y-%m-%d')
value.index = pd.to_datetime(value.index).strftime('%Y-%m-%d')
dividend.index = pd.to_datetime(dividend.index).strftime('%Y-%m-%d')
low_vol.index = pd.to_datetime(low_vol.index).strftime('%Y-%m-%d')
momentum.index = pd.to_datetime(momentum.index).strftime('%Y-%m-%d')


In [3]:
df = pd.concat([quality, value, dividend, low_vol, momentum], axis=1)
df.columns = ['Quality', 'Value', 'Dividend', 'Low Vol', 'Momentum']
df = df.dropna(how='any')

In [4]:
df

,Quality,Value,Dividend,Low Vol,Momentum
Date,,,,,
2020-01-03,-0.008130,-0.007121,-0.004342,0.000673,-0.003084
2020-01-06,0.001912,0.001619,0.003335,-0.001514,0.000738
2020-01-07,-0.003545,-0.003773,-0.002046,-0.000758,-0.005708
2020-01-08,0.005473,0.002628,0.001281,0.003372,0.005023
2020-01-09,0.007077,0.004471,0.001279,0.001260,0.010233
...,...,...,...,...,...
2023-12-21,0.009356,0.008913,0.009888,-0.005735,0.011688
2023-12-22,0.000927,0.004069,0.003350,0.003402,0.001849
2023-12-27,-0.000184,0.001900,0.001019,0.007297,0.003057


In [51]:
def f(w, cov_mat):
    s=0
    for i in range(len(w)):
        for j in range(len(w)):
            s += (w[i]*(cov_mat@w)[i] - w[j]*(cov_mat@w)[j])**2
    return s

cov_mat = df.cov().values
w0 = np.array([0.3, 0.2, 0.1, 0.2, 0.2])

#Minimize the function f(w, cov_mat) using the scipy.optimize.minimize function
constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1}, #sum of weights = 1
                {'type': 'ineq', 'fun': lambda w: w})   #weights must be positive (long only)
result = opt.minimize(f, w0, args=(cov_mat,), constraints=constraints, tol=1e-20)
w_opt = result.x

In [52]:
sigma = np.sqrt(w_opt.T@cov_mat@w_opt)

for i in range(5):
    s= (w_opt @ cov_mat[i]) / sigma
    print("Marginal contribution of factor {} : {}".format(i,w_opt[i]*s))

Marginal contribution of factor 0 : 0.002423167147104411
Marginal contribution of factor 1 : 0.0024238323301195752
Marginal contribution of factor 2 : 0.0024230824257962962
Marginal contribution of factor 3 : 0.002423312420137248
Marginal contribution of factor 4 : 0.0024229242751932483


In [10]:
pip install yesg

  Preparing metadata (setup.py) ... done
  Created wheel for yesg: filename=yesg-2.1.1-py3-none-any.whl size=6105 sha256=fdcef679672a94937fcdcfa1b1e163b603ec279d7ad9b35812cc40e7262a6f3d
  Stored in directory: /home/yohann/.cache/pip/wheels/cd/1c/41/4a3276b715ded8db45d25199e72be0190ea23afaef904e9b28
Successfully built yesg
Note: you may need to restart the kernel to use updated packages.


In [ ]:
#replace values from a list with a dictionary
df = pd.DataFrame({'A': [0, 1, 2, 3, 4], 'B': [5, 6, 7, 8, 9]})
d = {0: 'a', 1: 'b', 2: 'c', 3: 'd', 4: 'e'}
df = df.replace({'A': d, 'B': d})
df

In [12]:
import yesg

# get the s&p 500 tickers
tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]['Symbol'].tolist()
dico = {'GOOG':"GOOGL"}
df_esg = pd.DataFrame(index=tickers, columns=['ESG Score'])
for ticker in tickers:
    try:
        if(ticker in dico.keys()):
            df_esg.loc[ticker, "ESG Score"] = yesg.get_historic_esg(dico[ticker]).iloc[-1,0]
        else:
            df_esg.loc[ticker, "ESG Score"] = yesg.get_historic_esg(ticker).iloc[-1,0]
    except AttributeError:
        pass

An error has occurred. The ticker symbol might be wrong or you might need to wait to continue.
An error has occurred. The ticker symbol might be wrong or you might need to wait to continue.
An error has occurred. The ticker symbol might be wrong or you might need to wait to continue.
An error has occurred. The ticker symbol might be wrong or you might need to wait to continue.
An error has occurred. The ticker symbol might be wrong or you might need to wait to continue.
An error has occurred. The ticker symbol might be wrong or you might need to wait to continue.
An error has occurred. The ticker symbol might be wrong or you might need to wait to continue.
An error has occurred. The ticker symbol might be wrong or you might need to wait to continue.
An error has occurred. The ticker symbol might be wrong or you might need to wait to continue.
An error has occurred. The ticker symbol might be wrong or you might need to wait to continue.
An error has occurred. The ticker symbol might be 

In [20]:
df_esg[df_esg.isna().values].index

Index(['GOOG', 'AMCR', 'AMTM', 'BRK.B', 'BF.B', 'BLDR', 'CARR', 'CTLT', 'CRL',
       'CPAY', 'CTVA', 'DAY', 'DECK', 'DOW', 'EPAM', 'FICO', 'FOX', 'GEV',
       'GNRC', 'GDDY', 'DOC', 'HWM', 'INVH', 'JBL', 'KVUE', 'KKR', 'LHX', 'LW',
       'LYV', 'MOH', 'NWS', 'OTIS', 'PLTR', 'PAYC', 'SW', 'SOLV', 'STE', 'TDY',
       'TYL', 'UBER', 'VLTO', 'VICI', 'VST', 'WST'],
      dtype='object')

In [21]:
df_esg.dropna(axis=0, inplace=True)
#The best ESG score is 0
#drop the worst decile of the ESG scores (keep the top 90%)
quantile_threshold = 0.9
df_esg = df_esg[df_esg['ESG Score'] < df_esg['ESG Score'].quantile(quantile_threshold)]

### Implementations of equal risk contribution
- https://github.com/matthewgilbert/erc/blob/master/erc/erc.py
- https://github.com/mirca/riskparity.py (not used)
- https://thequantmba.wordpress.com/2016/12/14/risk-parityrisk-budgeting-portfolio-in-python/

### Papers
- [Paper of Maillard, Roncalli and Teiletche](http://thierry-roncalli.com/download/erc.pdf)
- [Slides of Maillard, Roncalli and Teiletche](http://www.thierry-roncalli.com/download/erc-slides.pdf)
- [Master's thesis of David Stefanovits](https://ethz.ch/content/dam/ethz/special-interest/math/risklab-dam/documents/walter-saxer-preis/ma-stefanovits.pdf)
